In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
session_times = pd.read_csv('../data/web_page_data.csv')
ax = session_times.boxplot(by= 'Page', column='Time')
session_times['Time'] = session_times['Time'].apply(lambda x: 100*x)
ax.set_xlabel('')
ax.set_ylabel('Time (in seconds)')
plt.suptitle('')

mean_a = session_times[session_times['Page']=='Page A'].Time.mean()
print(mean_a)
mean_b = session_times[session_times['Page']=='Page B'].Time.mean()
print(mean_b)
mean_b-mean_a


import random as rd
def perm_fun(x, nA, nB):
    n = nA + nB
    idx_B = set(rd.sample(range(n),nB))
    idx_A = set(range(n)) - idx_B
    idx_A = list(idx_A)
    idx_B = list(idx_B)
    return (x.loc[idx_B].mean() - x.loc[idx_A].mean())

perm_diffs = [perm_fun(session_times['Time'], 21, 15) for _ in range(1000)]
fig, ax  = plt.subplots(figsize = (5,5))
ax.hist(perm_diffs, bins=11,rwidth=0.9)
ax.axvline(x = mean_b- mean_a, color ='black', lw = 2)
ax.text(50, 190, 'Observed\ndifference',bbox = {'facecolor':'white'}, fontsize = 20)
ax.set_xlabel('Session time differences (in seconds)')
ax.set_ylabel('Frequency')
import numpy as np
perm_diffs = np.array([perm_diffs])
np.mean(perm_diffs> (mean_b-mean_a))

#관측된차이
obs_pct_diff = 100 * (200 /23739 - 182 / 22588)
print(f'관측된 차이 {obs_pct_diff:.4f}%')
#빈 상자 속에 0넣기
conversion = [0]*45945
#1넣기
conversion.extend([1]*382)
conversion = pd.DataFrame([conversion])
conversion=conversion.transpose()
conversion[0]
#1000번 conversion 재표본추출
perm_diffs = [100 * perm_fun(conversion[0], 23739, 22588) for _ in range(1000)]
fig, ax = plt.subplots(figsize = (5,5))
# #해당 결과 히스토그램 생성
plt.rcParams['font.family'] = 'D2Coding'
ax.hist(perm_diffs,bins = 11,rwidth=0.9)
ax.axvline(x= obs_pct_diff, color = 'black', lw= 2)
ax.text(0.06,200, '관측된\n오차',bbox= {'facecolor': 'white'})


np.mean([diff > obs_pct_diff for diff in perm_diffs])
import scipy.stats as stats
survivors = np.array([[200, 23739 - 200],[182, 22588 -182]])
chi2, p_value, df, _ = stats.chi2_contingency(survivors)
print(f'p-value 일원 분석:{p_value/2 :.4f}')
import numpy as np
import scipy.stats as stats
import pandas as pd
import matplotlib.pyplot as plt
session_times = pd.read_csv('../data/web_page_data.csv')
res = stats.ttest_ind(session_times[session_times.Page == 'Page A'].Time,
                      session_times[session_times.Page == 'Page B'].Time,
                      equal_var=False)
print(f'p-value for single sided test:{res.pvalue / 2:.4f}')
## ANOVA 분석
import pandas as pd
four_sessions = pd.DataFrame({'Page 1':[164,172,177,156,195,172],'Page 2':[178,191,182,185,177,185],'Page 3':[175,193,171,163,176,176],'Page 4':[155,166,164,170,168,162]})
four_sessions
observed_variance = four_sessions.groupby('Page').mean().var()[0]
print('Observed means:',four_sessions.groupby('Page').mean().values.ravel())
print('Variance', observed_variance)
def perm_test(df):
    df = df.copy()
    df['Time'] = np.random.permutation(df['Time'].values)
    return df.groupby('Page').mean().var()[0]
perm_variance = [perm_test(four_sessions) for _ in range(3000)]
print('Pr(Prop)',np.mean([var > observed_variance for var in perm_variance]))
## F 통계량
import statsmodels as sm
model = smf.ols('Time ~ Page', data = four_sessions).fit()
aov_table = sm.stats.anova_lm(model)
aov_table
import random
box = [1]*34
box.extend([0]*2966)
random.shuffle(box)

def chi2(observed, expected):
    pearson_residuals = []
    for row, expect in zip(observed, expected):
        pearson_residuals.append([(observe - expect)**2/expect for observe in row])
    return np.sum(pearson_residuals)
excepted_clicks = 34/3
excepted_noclicks = 1000 - excepted_clicks
excepted = [34/3, 1000-34/3]
chi2observed = chi2(clicks.values, excepted)

def perm_fun(box):
    sample_clicks = [sum(random.sample(box,1000)),
                     sum(random.sample(box,1000)),
                     sum(random.sample(box,1000))]
    sample_noclicks = [1000 - n for n in sample_clicks]
    return chi2([sample_clicks, sample_noclicks],expected)

perm_chi2 = [perm_fun(box) for _ in range(2000)]
resampled_p_value = sum(perm_chi2 > chi2observed) / len(perm_chi2)
print(chi2observed)
print(resampled_p_value)
import scipy.stats as stats
chisq, pvalue, df, excepted = stats.chi2_contingency(clicks)
chi2observed
pvalue
## 표본 크기 확인
import statsmodels as sm
effect_size = sm.stats.proportion_effectsize(0.0121,0.011)
analysis = sm.stats.TTestIndPower()
result = analysis.solve_power(effect_size = effect_size, alpha=0.05,power = 0.8, alternative = 'larger')
print('Sample Size: %.3f'%result)